In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
%matplotlib inline

In [3]:
# Importing the text the lazy way.
gatsby_orig="In my younger and more vulnerable years my father gave me some advice that I've been turning over in my mind ever since. \"Whenever you feel like criticizing any one,\" he told me, \"just remember that all the people in this world haven't had the advantages that you've had.\" He didn't say any more but we've always been unusually communicative in a reserved way, and I understood that he meant a great deal more than that. In consequence I'm inclined to reserve all judgments, a habit that has opened up many curious natures to me and also made me the victim of not a few veteran bores. The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men. Most of the confidences were unsought--frequently I have feigned sleep, preoccupation, or a hostile levity when I realized by some unmistakable sign that an intimate revelation was quivering on the horizon--for the intimate revelations of young men or at least the terms in which they express them are usually plagiaristic and marred by obvious suppressions. Reserving judgments is a matter of infinite hope. I am still a little afraid of missing something if I forget that, as my father snobbishly suggested, and I snobbishly repeat a sense of the fundamental decencies is parcelled out unequally at birth. And, after boasting this way of my tolerance, I come to the admission that it has a limit. Conduct may be founded on the hard rock or the wet marshes but after a certain point I don't care what it's founded on. When I came back from the East last autumn I felt that I wanted the world to be in uniform and at a sort of moral attention forever; I wanted no more riotous excursions with privileged glimpses into the human heart. Only Gatsby, the man who gives his name to this book, was exempt from my reaction--Gatsby who represented everything for which I have an unaffected scorn. If personality is an unbroken series of successful gestures, then there was something gorgeous about him, some heightened sensitivity to the promises of life, as if he were related to one of those intricate machines that register earthquakes ten thousand miles away. This responsiveness had nothing to do with that flabby impressionability which is dignified under the name of the \"creative temperament\"--it was an extraordinary gift for hope, a romantic readiness such as I have never found in any other person and which it is not likely I shall ever find again. No--Gatsby turned out all right at the end; it is what preyed on Gatsby, what foul dust floated in the wake of his dreams that temporarily closed out my interest in the abortive sorrows and short-winded elations of men."

# We want to use the standard english-language parser.
parser = spacy.load('en_core_web_md')

# Parsing Gatsby.
gatsby = parser(gatsby_orig)

# Dividing the text into sentences and storing them as a list of strings.
sentences=[]
for span in gatsby.sents:
    # go from the start to the end of each span, returning each token in the sentence
    # combine each token using join()
    sent = ''.join(gatsby[i].string for i in range(span.start, span.end)).strip()
    sentences.append(sent)

# Creating the tf-idf matrix.
counter = TfidfVectorizer(lowercase=False, 
                          stop_words=None,
                          ngram_range=(1, 1), 
                          analyzer=u'word', 
                          max_df=.5, 
                          min_df=1,
                          max_features=None, 
                          vocabulary=None, 
                          binary=False)

#Applying the vectorizer
data_counts=counter.fit_transform(sentences)

In [4]:
# Calculating similarity
similarity = data_counts * data_counts.T

# Identifying the sentence with the highest rank.
nx_graph = nx.from_scipy_sparse_matrix(similarity)
ranks=nx.pagerank(nx_graph, alpha=.85, tol=.00000001)

ranked = sorted(((ranks[i],s) for i,s in enumerate(sentences)),
                reverse=True)
print(ranked[0])

(0.06634505728672438, 'The abnormal mind is quick to detect and attach itself to this quality when it appears in a normal person, and so it came about that in college I was unjustly accused of being a politician, because I was privy to the secret griefs of wild, unknown men.')


In [50]:
# Modified code for bigrams
gat = []
for token in gatsby:
    if len(gatsby) - token.i == 1:
        break
    span = gatsby[token.i: token.i + 2]
    item_ = [token for token in span if (token.pos_ == 'NOUN' or token.pos_ == 'ADJ')]
    if len(item_) == 2:
        gat.append(span)
        
spans = set([span.text for span in gat])

adjacency=pd.DataFrame(columns=spans,index=spans,data=0)    

for i, word in enumerate(gatsby):
    span = gatsby[i:i + 2]
    if any([span == item for item in gat]):
        end=max(0,len(gatsby)-(len(gatsby)-(i+30)))
        nextwords = gatsby[i+1:end]
        nextwords_chk = []
        for i, tok in enumerate(nextwords):
            if len(nextwords) - i == 1:
                break
            sp = nextwords[i:i+2]
            nextwords_chk.append(sp)
        inset = [x in gat for x in nextwords_chk]
#         print(inset)
        span_neighbors = [nextwords_chk[i].text for i in range(len(nextwords_chk)) if inset[i]]
#         print([type(i.token) for i in span_neighbors])
        if span_neighbors:
            adjacency.loc[span.text,span_neighbors] = adjacency.loc[span.text,span_neighbors] + 1
print('done!')

done!


In [52]:
# Running TextRank
nx_words = nx.from_numpy_matrix(adjacency.as_matrix())
ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)

# Identifying the five most highly ranked keywords
ranked = sorted(((ranks[i],s) for i,s in enumerate(spans)),
                reverse=True)
print(ranked[:5])

[(0.03418122453632275, 'young men'), (0.033445952555567884, 'last autumn'), (0.03325348452281698, 'hostile levity'), (0.031117870191397588, 'many curious'), (0.031117870191397584, 'curious natures')]


/home/seapea/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


### Tricky exercise
Ignoring the fairly convoluted steps, we really can't expect the model to do much of anything with such a small corpus and bigrams. We had to use a large neighboring window to get any results (a window of 20 yielded all False inset values). WITH that said, the first two bigrams do look decent. I will try to incorporate this into my final capstone, as it is interesting.

In [53]:
# # Removing stop words and punctuation, then getting a list of all unique words in the text
# gatsby_filt = [word for word in gatsby if word.is_stop==False and (word.pos_=='NOUN' or word.pos_=='ADJ')]
# words = set(gatsby_filt)
# adjacency = pd.DataFrame(columns=words, index=words, data=0)

# for i, word in enumerate(gatsby):
#     if any([word == item for item in gatsby_filt]):
#         end = max(0, len(gatsby) - (len(gatsby) - (i + 5)))
#         nextwords = gatsby[i + 1:end]
#         inset = [x in gatsby_filt for x in nextwords]
#         neighbors = [nextwords[i] for i in range(len(nextwords)) if inset[i]]
#         print([type(i) for i in neighbors])
#         if neighbors:
#             adjacency.loc[word, neighbors] = adjacency.loc[word, neighbors] + 1
# adjacency.head()